The usual preliminaries....


In [1]:
pip install scikit-learn


In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")['train']

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
train_data = []
train_data_labels = []
for item in imdb_dataset:
  train_data.append(item['text'])
  train_data_labels.append(item['label'])

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word',max_features=1000,lowercase=True,stop_words='english',ngram_range=(1,2))
features = vectorizer.fit_transform(train_data).toarray()

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(features,train_data_labels,train_size=0.75,random_state=123)

We will use three models, Multinomial Naive Bayes, Random Forests and a Decision Tree to do the classification. Create the models.

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
model_nb = MultinomialNB()
model_dt = DecisionTreeClassifier()
model_rf = RandomForestClassifier()

Train the models.

In [11]:
model_nb = model_nb.fit(X=X_train,y=y_train)
model_dt = model_dt.fit(X=X_train,y=y_train)
model_rf = model_rf.fit(X=X_train,y=y_train)

Test the models on the validation set.

In [12]:
y_pred_nb = model_nb.predict(X_val)
y_pred_dt = model_dt.predict(X_val)
y_pred_rf = model_rf.predict(X_val)

Now let's calculate the accuracy of the models' predictions on the validation set.

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix
print("Naive Bayes", accuracy_score(y_val,y_pred_nb))
print(confusion_matrix(y_val,y_pred_nb))
print()
print("Decision Tree", accuracy_score(y_val,y_pred_dt))
print(confusion_matrix(y_val,y_pred_dt))
print()
print("Random Forest", accuracy_score(y_val,y_pred_rf))
print(confusion_matrix(y_val,y_pred_rf))


Naive Bayes 0.81728
[[2463  622]
 [ 520 2645]]

Decision Tree 0.70336
[[2208  877]
 [ 977 2188]]

Random Forest 0.81328
[[2541  544]
 [ 623 2542]]


Now create the voting ensemble...